In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_parquet('TRAIN_Reco_2021_2022_2023.parquet.gzip')
df.head()

,ID,high,low,close,volume
ExecutionTime,,,,,
2021-01-06 21:45:00+01:00,Fri00Q1,0.0,0.0,0.0,0.0
2021-01-06 22:00:00+01:00,Fri00Q1,0.0,0.0,0.0,0.0
2021-01-06 22:15:00+01:00,Fri00Q1,0.0,0.0,0.0,0.0
2021-01-06 22:30:00+01:00,Fri00Q1,0.0,0.0,0.0,0.0
2021-01-06 22:45:00+01:00,Fri00Q1,0.0,0.0,0.0,0.0


In [4]:
df.shape

(69805344, 5)

In [5]:
print("Memory usage:", df.memory_usage(deep=True).sum() / 1024**2, "MB")

Memory usage: 6390.8701171875 MB


In [6]:
float_columns = df.select_dtypes(include=['float64', 'float32']).columns
df[float_columns] = df[float_columns].astype(np.float16)

print("Memory usage after conversion:", df.memory_usage(deep=True).sum() / 1024**2, "MB")

Memory usage after conversion: 4793.152587890625 MB


In [7]:
df = df.reset_index()
# Convert 'ExecutionTime' to datetime
df['ExecutionTime'] = pd.to_datetime(df['ExecutionTime'])

print(df.dtypes)

ExecutionTime    datetime64[ns, Europe/Berlin]
ID                                      object
high                                   float16
low                                    float16
close                                  float16
volume                                 float16
dtype: object


In [8]:
print(df.isnull().sum())

ExecutionTime    0
ID               0
high             0
low              0
close            0
volume           0
dtype: int64


In [9]:
df.set_index('ExecutionTime', inplace=True)

In [10]:
df['ExecutionTime'] = pd.to_datetime(df['ExecutionTime'])
print(type(df['ExecutionTime']))
df.set_index('ExecutionTime', inplace=True)
df.head()

KeyError: 'ExecutionTime'

In [11]:
# Calculate the difference between consecutive timestamps
time_diffs = df.index.to_series().diff()

# Check the unique time differences
print(time_diffs.unique())


<TimedeltaArray>
[NaT, '0 days 00:15:00', '-1083 days +23:00:00']
Length: 3, dtype: timedelta64[ns]


In [12]:
# Identify rows where the time difference is not 15 minutes (including the large negative time jump)
irregular_intervals = df[time_diffs != pd.Timedelta('0 days 00:15:00')]
irregular_intervals.head()

C:\Users\a4293604\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pandas\io\formats\format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,ID,high,low,close,volume
ExecutionTime,,,,,
2021-01-06 21:45:00+01:00,Fri00Q1,0.0,0.0,0.0,0.0
2021-01-06 21:45:00+01:00,Fri00Q2,0.0,0.0,0.0,0.0
2021-01-06 21:45:00+01:00,Fri00Q3,0.0,0.0,0.0,0.0
2021-01-06 21:45:00+01:00,Fri00Q4,0.0,0.0,0.0,0.0
2021-01-06 21:45:00+01:00,Fri01Q1,0.0,0.0,0.0,0.0


In [13]:
irregular_intervals.shape

(672, 5)

In [14]:
df_cleaned = df.drop(irregular_intervals.index)
df_cleaned.shape


(69804672, 5)

In [15]:
# Calculate the difference between consecutive timestamps
time_diffs = df_cleaned.index.to_series().diff()

# Check the unique time differences
print(time_diffs.unique())


<TimedeltaArray>
[NaT, '0 days 00:15:00', '-1083 days +23:15:00']
Length: 3, dtype: timedelta64[ns]


In [54]:
irregular_intervals = irregular_intervals.reset_index()

In [55]:
irregular_intervals.head()

C:\Users\a4293604\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pandas\io\formats\format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
C:\Users\a4293604\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pandas\io\formats\format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,ExecutionTime,ID,high,low,close,volume
0,2021-01-06 21:45:00+01:00,Fri00Q1,0.0,0.0,0.0,0.0
1,2021-01-06 21:45:00+01:00,Fri00Q2,0.0,0.0,0.0,0.0
2,2021-01-06 21:45:00+01:00,Fri00Q3,0.0,0.0,0.0,0.0
3,2021-01-06 21:45:00+01:00,Fri00Q4,0.0,0.0,0.0,0.0
4,2021-01-06 21:45:00+01:00,Fri01Q1,0.0,0.0,0.0,0.0


In [56]:
# Set ExecutionTime and ID as a MultiIndex
irregular_intervals.set_index(['ExecutionTime', 'ID'], inplace=True)

# Now you can safely perform operations without duplicate issues
print(irregular_intervals.head())


                                   high  low  close  volume
ExecutionTime             ID                               
2021-01-06 21:45:00+01:00 Fri00Q1   0.0  0.0    0.0     0.0
                          Fri00Q2   0.0  0.0    0.0     0.0
                          Fri00Q3   0.0  0.0    0.0     0.0
                          Fri00Q4   0.0  0.0    0.0     0.0
                          Fri01Q1   0.0  0.0    0.0     0.0


C:\Users\a4293604\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pandas\io\formats\format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


In [60]:
# Resample on ExecutionTime, applying aggregation to each contract individually
data_resampled = irregular_intervals.groupby(level='ExecutionTime').resample('15T', level=0).mean()

# You can choose aggregation methods like mean, sum, or other operations if needed
print(data_resampled.head())


                                                          high        low  \
ExecutionTime             ExecutionTime                                     
2021-01-06 21:45:00+01:00 2021-01-06 21:45:00+01:00  11.403786  10.963588   

                                                         close   volume  
ExecutionTime             ExecutionTime                                  
2021-01-06 21:45:00+01:00 2021-01-06 21:45:00+01:00  11.232887  0.18544  


C:\Users\a4293604\AppData\Local\Temp\ipykernel_25564\613765840.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  data_resampled = irregular_intervals.groupby(level='ExecutionTime').resample('15T', level=0).mean()


In [ ]:
# # Pivot the data to create separate columns for each contract ID's high, low, close, and volume
# data_pivoted = irregular_intervals.pivot_table(index='ExecutionTime', columns='ID', values=['high', 'low', 'close', 'volume'])

# # Flatten the multi-level columns for easier access
# data_pivoted.columns = ['_'.join(col).strip() for col in data_pivoted.columns.values]

# print(data_pivoted.head())


In [62]:
# Check for missing intervals
time_diffs = data_resampled.index.to_series().diff()
missing_intervals = data_resampled[time_diffs != pd.Timedelta('0 days 00:15:00')]
(missing_intervals)


,,high,low,close,volume
ExecutionTime,ExecutionTime,,,,
2021-01-06 21:45:00+01:00,2021-01-06 21:45:00+01:00,11.403786,10.963588,11.232887,0.18544


In [24]:
# Resample to 15-minute intervals
data_resampled = irregular_intervals.groupby('ID').resample('15min').mean()
data_resampled = data_resampled.reset_index()

# Fill missing values (forward fill, backward fill, or interpolation)
data_resampled_filled = data_resampled.fillna(method='ffill', axis=0, inplace=False)

C:\Users\a4293604\AppData\Local\Temp\ipykernel_28336\1254555075.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_resampled_filled = data_resampled.fillna(method='ffill', axis=0, inplace=False)


TypeError: No matching signature found

In [28]:
df=df.reset_index()
df.head()

MemoryError: Unable to allocate 533. MiB for an array with shape (1, 69805344) and data type object

In [26]:
# Handle duplicate 'ExecutionTime' by adding a small offset to create unique timestamps
df['ExecutionTime'] += pd.to_timedelta(df.groupby('ExecutionTime').cumcount(), unit='ms')

# Set 'ExecutionTime' as index
df.set_index('ExecutionTime', inplace=True)

# Resample to 15-minute intervals
resampled_df = df.resample('15T').asfreq()

# Interpolating numeric columns
resampled_df[['high', 'low', 'close', 'volume']] = resampled_df[['high', 'low', 'close', 'volume']].interpolate()

# Forward fill the ID column (or use another method based on context)
resampled_df['ID'] = resampled_df['ID'].fillna(method='ffill')

# Reset index to make it more usable
resampled_df.reset_index(inplace=True)

# Display the result
print(resampled_df)

KeyError: 'ExecutionTime'